# Step 4: Scenario Discovery
#### MORDM - Evaluating Multi-Disease Interventions - MSc Engineering and Policy Analysis

Shannon M. Gross

In [1]:
# standard packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import time, copy
import seaborn as sns
# import plotly.express as px
pd.set_option('display.max_columns', 100)

# EMA imports
from ema_workbench.connectors.vensim import VensimModel
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress) 
from ema_workbench import (SequentialEvaluator, MultiprocessingEvaluator, Policy, Scenario, Constraint, CategoricalParameter,
                           TimeSeriesOutcome, ScalarOutcome, IntegerParameter, RealParameter, save_results, load_results, Model)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.parameters import create_parameters
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.analysis import pairs_plotting, plotting, plotting_util, feature_scoring, parcoords, prim, dimensional_stacking
from ema_workbench.analysis.plotting import lines, Density
from ema_workbench.util import ema_logging
ema_logging.log_to_stderr(ema_logging.INFO)

# problem-specific imports
from disease_model_problems import get_model_for_problem_formulation 

C:\Users\shannonsgross\Anaconda3\envs\thesis\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Users\shannonsgross\Anaconda3\envs\thesis\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


### Retrieve robust results

In [2]:
# %matplotlib notebook
for PF in range(4,5):
    print('regret from Problem Formulation {} '.format(PF) )
    file_name = r'.\results\robust_candidates\regret\PF{}'.format(PF) + '.tar.gz'
    pareto_results = load_results(file_name)
    experiments, outcomes = pareto_results
    disease_model = get_model_for_problem_formulation(PF)   
#     #test w small subset -avoid memory error
#     df_out = pd.DataFrame(outcomes).head(3800)
#     df_out.fillna(0,inplace=True)  
#     df_exp = pd.read_csv(file_name, compression='gzip', sep=',', nrows=3800)
#     df_exp.fillna(0,inplace=True)
#     small_exp = df_exp.drop(labels=[l.name for l in disease_model.levers], axis=1)
#     small_exp2 = small_exp.drop([ 'policy','Unnamed: 0','experiments.csv'], axis=1)        

    cleaned_exp = experiments.drop(labels=[l.name for l in disease_model.levers], axis=1)
    cleaned_exp2 = cleaned_exp.drop([ 'policy','Unnamed: 0'], axis=1)

regret from Problem Formulation 1 


[MainProcess/INFO] results loaded succesfully from C:\Users\shannonsgross\Documents\thesis\disease_model\results\RobustCandidates\PF1.tar.gz


#### Where do policies fail concerning Mortality objective?

In [ ]:
ooi = 'Mortality'

x = cleaned_exp2 #independent variable experiments
y = outcomes[ooi] > np.percentile(outcomes[ooi], 10) 
    #focus on 10 percent of the worst outcomes for this objective 
prim_alg = prim.Prim(x, y, threshold=0.8)
box_mort = prim_alg.find_box()

box_mort.show_tradeoff()
plt.title("Peeling Trajectory for *{}* under PF {}".format(ooi,PF))
plt.figure(figsize=(4,4)) 
plt.show()

In [ ]:
boi=2
box_mort.inspect(boi, style='graph')
print(prim_alg.stats_to_dataframe())
# box1.show_ppt()

In [ ]:
box_mort.show_pairs_scatter(boi)

In [ ]:
# based on the peeling trajectory, we pick entry number 44
box_mort.select(boi)
# show the resulting box
prim_alg.show_boxes(boi)
prim_alg.boxes_to_dataframe()

#### Where do policies fail to meet Morbidity objective?

In [ ]:
ooi = 'Morbidity'

data = outcomes[ooi]    
y = data > np.percentile(data, 10) 
    #focus on 10 percent of the worst outcomes for this objective   
prim_alg = prim.Prim(cleaned_exp2, y, threshold=0.8)
box_morb = prim_alg.find_box()  

box_morb.show_tradeoff()
plt.title("Peeling Trajectory for *{}* under PF {}".format(ooi,PF))
plt.figure(figsize=(4,4)) 
plt.show()

In [ ]:
boi=3

box_morb.inspect(boi, style='graph')
print(prim_alg.stats_to_dataframe())

In [ ]:
box_morb.show_pairs_scatter(boi)

#### Where do policies fail concerning Timeliness (prevalence reduction) objective?

In [ ]:
ooi = 'Timeliness'

data = outcomes[ooi]    
y = data > np.percentile(data, 10) 
    #focus on 10 percent of the worst outcomes for this objective   
prim_alg = prim.Prim(cleaned_exp2, y, threshold=0.8)
box_prev = prim_alg.find_box()  

box_prev.show_tradeoff()
plt.title("Peeling Trajectory for *{}* under PF {}".format(ooi,PF))
plt.figure(figsize=(4,4)) 
plt.show()

In [ ]:
boi=3

box_prev.inspect(boi, style='graph')
print(prim_alg.stats_to_dataframe())

In [ ]:
box_prev.show_pairs_scatter(boi)

In [ ]:
#try dim stacking

from ema_workbench.analysis import dimensional_stacking
data = outcomes['Timeliness'] 
y = data > np.percentile(data, 90)

dimensional_stacking.create_pivot_plot(cleaned_exp2, y)
plt.show()